In [1]:
#https://python.plainenglish.io/python-pull-gcp-cloud-storage-file-into-aws-s3-6f6cb8c144b2
# Install the google-cloud-bigquery and pydata-google-auth packages
# to use this code snippet.
import google.cloud.bigquery
import google.cloud.bigquery.magics
#import pydata_google_auth
#for google storage
from google.cloud import storage
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
from google.oauth2 import service_account

In [2]:
import pandas as pd
import io, numpy as np, re, os, sys, json

In [3]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="generated-atlas-345415-ee603c499617.json"

In [4]:
#local variables for credentials
service_account_json = os.environ["GOOGLE_APPLICATION_CREDENTIALS"]
#Set your GCP project Name
BgProject="Snow Cast"
#Construct a BigQuery client object.
bq_client = bigquery.Client()
gcpbucket = storage.Client().get_bucket("snowcast_maindata")
#set the your S3 bucket name
s3bucket="my-aws-s3-bucket-name"
#Configure the magics to use your credentials and project.
google.cloud.bigquery.magics.context.project = BgProject
google.cloud.bigquery.magics.context.credentials = service_account_json
credentials = service_account.Credentials.from_service_account_file(service_account_json)
client = bigquery.Client(credentials= credentials,project=BgProject)

In [5]:
#this function will be returned dataframe 
#from gcp google storage file
# gcpbucket: gcp bucket name, filename: gcp file name
def load_gsdatafile(gcpbucket, filename):
    error='-1'
    try: 
        bucket=gcpbucket.name
        filename = filename
        gsuri = 'gs://{}/{}'.format(bucket, filename) 
        df = pd.read_csv(gsuri,  encoding='utf-8')
        df.head()
        return df
    except Exception as e:
        return error

In [6]:
#this function will load dataframe into S3 bucket
#df: dataframe, s3bucket: S3 bucket name, file_key: file name
def savefile_toS3(df, s3bucket, file_key):
    msg='-1'
    try: 
        bucket=s3bucket
        file_key = file_key
        s3uri = 's3://{}/{}'.format(bucket, file_key) 
        print(s3uri) 
        df.to_csv(s3uri)
        msg='Files has been saved successfully!'
        return msg
    except Exception as e:
        return msg

In [7]:
filepath = "GRIDMET_Weather_Data.csv"

In [8]:
gdf=load_gsdatafile(gcpbucket,filepath)

In [9]:
gdf

,cell_id,geometry,date,precip_daily,wind_dir_avg,temp_min,temp_max,wind_vel
0,ASO_50M_SWE_USCAJW_3012,POLYGON ((-119.3966606691139 37.58805474778629...,2019-03-12,0.0,114.00,262.80,275.20,5.00
1,ASO_50M_SWE_USCAJW_3012,POLYGON ((-119.3966606691139 37.58805474778629...,2019-04-18,0.0,146.00,268.90,286.20,3.80
2,ASO_50M_SWE_USCAJW_3012,POLYGON ((-119.3966606691139 37.58805474778629...,2019-04-14,0.0,248.00,268.90,285.50,3.70
3,ASO_50M_SWE_USCAJW_3012,POLYGON ((-119.3966606691139 37.58805474778629...,2016-07-05,0.0,215.42,279.89,294.03,4.00
4,ASO_50M_SWE_USCAJW_3012,POLYGON ((-119.3966606691139 37.58805474778629...,2017-01-25,0.0,196.00,255.50,267.50,3.50
...,...,...,...,...,...,...,...,...
5380131,ASO_50M_SWE_USCATB_2267,POLYGON ((-119.31499791313145 37.8226861044099...,2018-05-20,0.0,182.00,271.20,283.10,2.20
5380132,ASO_50M_SWE_USCATB_2267,POLYGON ((-119.31499791313145 37.8226861044099...,2019-05-27,11.0,233.00,268.70,272.10,5.70
5380133,ASO_50M_SWE_USCATB_2267,POLYGON ((-119.31499791313145 37.8226861044099...,2017-08-07,0.4,182.00,278.30,287.40,2.40
5380134,ASO_50M_SWE_USCATB_2267,POLYGON ((-119.31499791313145 37.8226861044099...,2019-07-05,0.0,219.00,271.10,288.80,1.80


In [11]:
os.getcwd()

'/home/ubuntu/2022_snowpack_capstone/notebooks/data_ingestion'

In [12]:
gdf.to_csv(f'/home/ubuntu/SnowData/{filepath}',index=False)